### **Funciones**

Este Notebook tiene como objetivo crear pequeños DataSets a partir de los datos procesados, con la finalidad de optimizar el trabajo de las funciones y evitar problemas de memoria en render.

### **Librerias**

In [76]:
import pandas as pd

### **Extracción y lectura de datos preparados**

In [77]:
# lectura de archivos 
df_steam_games=pd.read_csv('ETL/steam_games.csv')
df_user_items=pd.read_csv('ETL/user_items.csv')
df_user_reviews=pd.read_csv('ETL/user_reviews.csv')

### **Exploración**

In [78]:
df_user_items[df_user_items['item_id']==761140]

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks


In [79]:
df_steam_games['item_id'].nunique()

28760

In [80]:
df_user_items['item_id'].nunique()

10978

In [81]:
# Conclusion: El DataFrame steam games contiene todos los juegos que existe, mientras que el data frame user items solo tiene datos de los juegos qeu fueron jugados por los usuarios


In [82]:
df_user_reviews.head(3)

,user_id,item_id,recommend,review_posted,year_review_posted,sentiment_analysis
0,76561197970982479,1250,True,2011-11-05,2011,2
1,76561197970982479,22200,True,2011-07-15,2011,2
2,76561197970982479,43110,True,2011-04-21,2011,2


### **Funcion 1** ______________________________________________________________________________________________________________________________________________________________________________

#### **Creación de DataSet optimizado**

En esta etapa se elaborará un data set optimizado con las columnas y filas necesarias para el desempeño de la funcion 1

In [83]:
# Union de los DataFrames que contienen los campos necesarios para el desempeño de la funcion 1 
df_funcion1=pd.merge(df_user_items,df_steam_games,on='item_id',how='left')
df_funcion1

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks,publisher,genres,app_name,release_date,price,developer,release_year
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0,0.0,Valve,Action,Counter-Strike,2000-11-01,9.99,Valve,2000.0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0,0.0,Valve,Action,Team Fortress Classic,1999-04-01,4.99,Valve,1999.0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0,0.0,Valve,Action,Day of Defeat,2003-05-01,4.99,Valve,2003.0
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0,0.0,Valve,Action,Deathmatch Classic,2001-06-01,4.99,Valve,2001.0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0,Valve,Action,Half-Life: Opposing Force,1999-11-01,4.99,Gearbox Software,1999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10685727,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0,CoaguCo Industries,Free To Play,One Way To Die: Steam Edition,2015-09-01,0.00,CoaguCo Industries,2015.0
10685728,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0,Tamationgames,Casual,You Have 10 Seconds 2,2016-08-24,0.00,Tamationgames,2016.0
10685729,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0,Tamationgames,Free To Play,You Have 10 Seconds 2,2016-08-24,0.00,Tamationgames,2016.0
10685730,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0,Tamationgames,Indie,You Have 10 Seconds 2,2016-08-24,0.00,Tamationgames,2016.0


In [84]:

# Seleccionala las columnas reelevantes para la funcion 1 , mostrando la suma de el tiempo jugado por los usuaarios (playtime_forever) agrupando los 
# mismos por genero (genres) y año de lanzamiento (release_year).

df_funcion1=df_funcion1.loc[:,['genres','release_year','playtime_forever']].groupby(by=['genres','release_year']).sum()
df_funcion1


playtime_forever
genres         release_year                  
Action         1983.0                  3473.0
               1984.0                   384.0
               1988.0                 16001.0
               1989.0                   607.0
               1990.0                 18271.0
...                                       ...
Web Publishing 2013.0                314963.0
               2014.0                 33641.0
               2015.0                348673.0
               2016.0                     1.0
               2017.0                  9382.0

[352 rows x 1 columns]

In [85]:
# Coloca los inidices como parte del data frame
df_funcion1.reset_index(inplace=True)

# Data Frame optimizado para la funcion 1 
df_funcion1

,genres,release_year,playtime_forever
0,Action,1983.0,3473.0
1,Action,1984.0,384.0
2,Action,1988.0,16001.0
3,Action,1989.0,607.0
4,Action,1990.0,18271.0
...,...,...,...
347,Web Publishing,2013.0,314963.0
348,Web Publishing,2014.0,33641.0
349,Web Publishing,2015.0,348673.0
350,Web Publishing,2016.0,1.0


In [86]:
# Creo un Dataset reducido con los datos necesarios para ser consumido por la API 
df_funcion1.to_csv('funcion1.csv',index=False) 

#### **Desarrollo de la funcion 1**

In [87]:
def PlayTimeGenre( genre : str ): #  Debe devolver año con mas horas jugadas para dicho género.
    # pasar a dataframe dentro de la funcion ?
    try:
        genre=genre.title()
        valor_maximo=df_funcion1[df_funcion1['genres']==genre]['playtime_forever'].max()
        indice=df_funcion1[df_funcion1['playtime_forever']==valor_maximo].index
        resultado=df_funcion1['release_year'].loc[indice].values
        return {f'Año de lanzamiento con mas horas jugadas para el Género {genre}:':resultado[0]}
    except Exception as e:
        print('Genero incorrecto')


# Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

### **Funcion 2** ______________________________________________________________________________________________________________________________________________________________________________

In [88]:
# union de los data frames necesarios para la funcion 2 
df_funcion2=pd.merge(df_user_items,df_steam_games,on='item_id',how='left').merge(df_user_reviews,on=['user_id','item_id'],how='left')
df_funcion2

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks,publisher,genres,app_name,release_date,price,developer,release_year,recommend,review_posted,year_review_posted,sentiment_analysis
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0,0.0,Valve,Action,Counter-Strike,2000-11-01,9.99,Valve,2000.0,NaN,NaN,NaN,NaN
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0,0.0,Valve,Action,Team Fortress Classic,1999-04-01,4.99,Valve,1999.0,NaN,NaN,NaN,NaN
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0,0.0,Valve,Action,Day of Defeat,2003-05-01,4.99,Valve,2003.0,NaN,NaN,NaN,NaN
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0,0.0,Valve,Action,Deathmatch Classic,2001-06-01,4.99,Valve,2001.0,NaN,NaN,NaN,NaN
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0,Valve,Action,Half-Life: Opposing Force,1999-11-01,4.99,Gearbox Software,1999.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10685727,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0,CoaguCo Industries,Free To Play,One Way To Die: Steam Edition,2015-09-01,0.00,CoaguCo Industries,2015.0,NaN,NaN,NaN,NaN
10685728,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0,Tamationgames,Casual,You Have 10 Seconds 2,2016-08-24,0.00,Tamationgames,2016.0,NaN,NaN,NaN,NaN
10685729,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0,Tamationgames,Free To Play,You Have 10 Seconds 2,2016-08-24,0.00,Tamationgames,2016.0,NaN,NaN,NaN,NaN
10685730,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0,Tamationgames,Indie,You Have 10 Seconds 2,2016-08-24,0.00,Tamationgames,2016.0,NaN,NaN,NaN,NaN


In [89]:
# Seleccion de columnas reelevantes
df_funcion2=df_funcion2.loc[:,['user_id','genres','year_review_posted','playtime_forever']] 
df_funcion2

,user_id,genres,year_review_posted,playtime_forever
0,76561197970982479,Action,NaN,6.0
1,76561197970982479,Action,NaN,0.0
2,76561197970982479,Action,NaN,7.0
3,76561197970982479,Action,NaN,0.0
4,76561197970982479,Action,NaN,0.0
...,...,...,...,...
10685727,76561198329548331,Free To Play,NaN,3.0
10685728,76561198329548331,Casual,NaN,4.0
10685729,76561198329548331,Free To Play,NaN,4.0
10685730,76561198329548331,Indie,NaN,4.0


In [90]:
df_funcion2.isnull().sum()    

# Observacion: Para obtener el dato de horas jugadas por año, es importante recalcar que no existe un dato certero respecto de las fechas en las que el usuario jugó , por tal motivo 
# se decide adoptar como fecha patron a la fecha donde el usuario efectuo una reseña sobre el juego, esta decicion implica que muchos usuarios carezcan de datos de los años 
# con mas horas jugadas para un determinado genero, esto se debe a que no todos los usuarios que jugaron un juego realizaron una reseña.

# Por otro lado,en el data Frame resultante encontramos registros donde no existen datos del Genero (genres), esto puede deberse a que en el proceso de limpieza y transformacion de datos 
# Se perdieron algunos registros, por lo cual , los valores que no posean datos en la columna genres tambien serán eliminados dado que para la funcion en desarrollo no son reelevantes 
# aquellos datos que carezcan de genero dado qeu esete es el dato que debe pasarse como parametro a la funcion en desarrollo. 

# Por lo expuesto se decide trabajar unicamente con aquellos registros que esten completos, eliminando asi todos aquellos registros que carezcan de datos en el campo "year_review_posted"
# y en el campo "genres"

user_id                      0
genres                 1041351
year_review_posted    10602218
playtime_forever             0
dtype: int64

In [91]:
# Elimino registros que contengan valores nulos en los campos 'year_review_posted' y "genres"
df_funcion2=df_funcion2.dropna(subset=['year_review_posted','genres'])
df_funcion2


,user_id,genres,year_review_posted,playtime_forever
50,76561197970982479,Action,2011.0,271.0
51,76561197970982479,Indie,2011.0,271.0
55,76561197970982479,Action,2011.0,10006.0
1006,js41637,Indie,2013.0,551.0
1007,js41637,Simulation,2013.0,551.0
...,...,...,...,...
10674375,76561198239215706,Action,2015.0,4659.0
10675255,wayfeng,Action,2015.0,42740.0
10675392,76561198251004808,Rpg,2015.0,1098.0
10676560,72947282842,Action,2015.0,33.0


In [92]:
# Visualizar los datos de la sumatoria de horas jugadas, agrupadas por genero, usuario y año
df_funcion2.groupby(by=['genres','user_id','year_review_posted']).sum()
# Resetear el indice para el nuevo DataFrame obtenido
df_funcion2=df_funcion2.reset_index(drop=True)
df_funcion2

,user_id,genres,year_review_posted,playtime_forever
0,76561197970982479,Action,2011.0,271.0
1,76561197970982479,Indie,2011.0,271.0
2,76561197970982479,Action,2011.0,10006.0
3,js41637,Indie,2013.0,551.0
4,js41637,Simulation,2013.0,551.0
...,...,...,...,...
78359,76561198239215706,Action,2015.0,4659.0
78360,wayfeng,Action,2015.0,42740.0
78361,76561198251004808,Rpg,2015.0,1098.0
78362,72947282842,Action,2015.0,33.0


#### **Exportacion de DataSet resultante**

In [93]:
# Exportar datos a un archivo csv, el cual será consumido por la API
df_funcion2.to_csv('funcion2.csv',index=False)

#### **Desarrollo de la función**

In [94]:
df_funcion2.head(3)

,user_id,genres,year_review_posted,playtime_forever
0,76561197970982479,Action,2011.0,271.0
1,76561197970982479,Indie,2011.0,271.0
2,76561197970982479,Action,2011.0,10006.0


In [95]:
def UserForGenre( genero : str ): #Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
    try:
        genero=genero.title() # independientemente de si el usuario coloca el parametro en mayuscula o minuscula, el texto introducido se convierte a un texto con la primer letra en mayusculas y las otrwas en minusculas
        df_genero=df_funcion2[df_funcion2['genres']==genero]
        tiempo_maximo=df_genero['playtime_forever'].max()
        indice_tiempo_maximo=df_genero[df_genero['playtime_forever']==tiempo_maximo].index
        resultado=df_genero['user_id'].loc[indice_tiempo_maximo].values[0]

        df_horas_por_año=df_funcion2[(df_funcion2['user_id']==resultado) & (df_funcion2['genres']==genero)]
        lista=[]
        for indice,fila in df_horas_por_año.iterrows():
            año=fila['year_review_posted']
            horas_jugadas=fila['playtime_forever']
            lista.append({'Año': año, 'Horas': horas_jugadas})

        return {f'Usuario con más horas jugadas para el Género {genero}':resultado, 'Horas jugadas': lista}

    except Exception as e:
        return {'Genero incorrecto'}

# Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}



##### **Funcion 3** ______________________________________________________________________________________________________________________________________________________________________________

In [149]:
df_funcion3=pd.merge(df_user_reviews,df_steam_games,on='item_id',how='left').merge(df_user_items,on=['user_id','item_id'],how='left')
df_funcion3

,user_id,item_id,recommend,review_posted,year_review_posted,sentiment_analysis,publisher,genres,app_name,release_date,price,developer,release_year,items_count,steam_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,1250,True,2011-11-05,2011,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,19.99,Tripwire Interactive,2009.0,277.0,7.656120e+16,Killing Floor,10006.0,0.0
1,76561197970982479,22200,True,2011-07-15,2011,2,ACE Team,Action,Zeno Clash,2009-04-21,9.99,ACE Team,2009.0,277.0,7.656120e+16,Zeno Clash,271.0,0.0
2,76561197970982479,22200,True,2011-07-15,2011,2,ACE Team,Indie,Zeno Clash,2009-04-21,9.99,ACE Team,2009.0,277.0,7.656120e+16,Zeno Clash,271.0,0.0
3,76561197970982479,43110,True,2011-04-21,2011,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,277.0,7.656120e+16,Metro 2033,834.0,0.0
4,js41637,251610,True,2014-06-24,2014,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,888.0,7.656120e+16,Barbie™ Dreamhouse Party™,84.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108504,wayfeng,730,True,2015-10-14,2015,1,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,14.99,Valve,2012.0,48.0,7.656120e+16,Counter-Strike: Global Offensive,42740.0,270.0
108505,76561198251004808,253980,True,2015-10-10,2015,2,Topware Interactive,Rpg,Enclave,2003-03-10,4.99,Starbreeze,2003.0,20.0,7.656120e+16,Enclave,1098.0,0.0
108506,72947282842,730,True,2015-10-31,2015,0,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,14.99,Valve,2012.0,2.0,7.656120e+16,Counter-Strike: Global Offensive,33.0,0.0
108507,ApxLGhost,730,True,2015-12-14,2015,2,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,14.99,Valve,2012.0,8.0,7.656120e+16,Counter-Strike: Global Offensive,10121.0,0.0


In [150]:
# Selecciono los campos reelevantes para mi funcion 
df_funcion3=df_funcion3.loc[:,['item_id','app_name','item_name','recommend','year_review_posted','sentiment_analysis']]
df_funcion3

,item_id,app_name,item_name,recommend,year_review_posted,sentiment_analysis
0,1250,Killing Floor,Killing Floor,True,2011,2
1,22200,Zeno Clash,Zeno Clash,True,2011,2
2,22200,Zeno Clash,Zeno Clash,True,2011,2
3,43110,NaN,Metro 2033,True,2011,2
4,251610,NaN,Barbie™ Dreamhouse Party™,True,2014,2
...,...,...,...,...,...,...
108504,730,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive,True,2015,1
108505,253980,Enclave,Enclave,True,2015,2
108506,730,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive,True,2015,0
108507,730,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive,True,2015,2


In [151]:
# filtro los registros obteniendo el siguiente dataframe
df_funcion3=df_funcion3[(df_funcion3['recommend']==True) & ((df_funcion3['sentiment_analysis']==2) | (df_funcion3['sentiment_analysis']==1))]
df_funcion3

,item_id,app_name,item_name,recommend,year_review_posted,sentiment_analysis
0,1250,Killing Floor,Killing Floor,True,2011,2
1,22200,Zeno Clash,Zeno Clash,True,2011,2
2,22200,Zeno Clash,Zeno Clash,True,2011,2
3,43110,NaN,Metro 2033,True,2011,2
4,251610,NaN,Barbie™ Dreamhouse Party™,True,2014,2
...,...,...,...,...,...,...
108503,730,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive,True,2015,1
108504,730,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive,True,2015,1
108505,253980,Enclave,Enclave,True,2015,2
108507,730,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive,True,2015,2


In [152]:
df_funcion3.isnull().sum()

item_id                   0
app_name               5630
item_name             18561
recommend                 0
year_review_posted        0
sentiment_analysis        0
dtype: int64

In [153]:
# completo los nombres faltantes en app_name por los que figuran en item_name
df_funcion3['app_name'].fillna(df_funcion3['item_name'],inplace=True)
# Una vez reemplazados los datos elimino columna item_name'
df_funcion3.drop('item_name',axis=1,inplace=True)
df_funcion3

C:\Users\pablo\AppData\Local\Temp\ipykernel_4212\4021196631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_funcion3['app_name'].fillna(df_funcion3['item_name'],inplace=True)
C:\Users\pablo\AppData\Local\Temp\ipykernel_4212\4021196631.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_funcion3.drop('item_name',axis=1,inplace=True)


,item_id,app_name,recommend,year_review_posted,sentiment_analysis
0,1250,Killing Floor,True,2011,2
1,22200,Zeno Clash,True,2011,2
2,22200,Zeno Clash,True,2011,2
3,43110,Metro 2033,True,2011,2
4,251610,Barbie™ Dreamhouse Party™,True,2014,2
...,...,...,...,...,...
108503,730,Counter-Strike: Global Offensive,True,2015,1
108504,730,Counter-Strike: Global Offensive,True,2015,1
108505,253980,Enclave,True,2015,2
108507,730,Counter-Strike: Global Offensive,True,2015,2


In [154]:
df_funcion3.isnull().sum()

item_id                  0
app_name              2129
recommend                0
year_review_posted       0
sentiment_analysis       0
dtype: int64

In [155]:
# Elimino registros cuyos valores presentes en el campo "app_name" sean nulos 

df_funcion3.dropna(subset='app_name',inplace=True)
df_funcion3

C:\Users\pablo\AppData\Local\Temp\ipykernel_4212\2856699192.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_funcion3.dropna(subset='app_name',inplace=True)


,item_id,app_name,recommend,year_review_posted,sentiment_analysis
0,1250,Killing Floor,True,2011,2
1,22200,Zeno Clash,True,2011,2
2,22200,Zeno Clash,True,2011,2
3,43110,Metro 2033,True,2011,2
4,251610,Barbie™ Dreamhouse Party™,True,2014,2
...,...,...,...,...,...
108503,730,Counter-Strike: Global Offensive,True,2015,1
108504,730,Counter-Strike: Global Offensive,True,2015,1
108505,253980,Enclave,True,2015,2
108507,730,Counter-Strike: Global Offensive,True,2015,2


In [156]:
# Exploracion de los datos resultantes 

# Se observa a continuacion qeu el data frame resultante contiene datos nulos en la columna app_name (columna donde se muestra el nombre del juego),
# esto se debe probablemente a que durante el proceso de limpieza 
# y transformacion del data frame steam_games se perdieron algunos registros. 

# Por lo tanto dichos registros se eliminaran por cuostiones del data frame que uitilizaremos para la funcion 3 , dado que la funcion debe devolver el nombre  

df_funcion3.isnull().sum()


item_id               0
app_name              0
recommend             0
year_review_posted    0
sentiment_analysis    0
dtype: int64

In [157]:
df_funcion3=df_funcion3.loc[:,['item_id','app_name','recommend','year_review_posted']]
df_funcion3

,item_id,app_name,recommend,year_review_posted
0,1250,Killing Floor,True,2011
1,22200,Zeno Clash,True,2011
2,22200,Zeno Clash,True,2011
3,43110,Metro 2033,True,2011
4,251610,Barbie™ Dreamhouse Party™,True,2014
...,...,...,...,...
108503,730,Counter-Strike: Global Offensive,True,2015
108504,730,Counter-Strike: Global Offensive,True,2015
108505,253980,Enclave,True,2015
108507,730,Counter-Strike: Global Offensive,True,2015


In [158]:
df_funcion3=df_funcion3.groupby(by=['item_id','app_name','year_review_posted']).count()
df_funcion3.reset_index(inplace=True)
df_funcion3

,item_id,app_name,year_review_posted,recommend
0,10,Counter-Strike,2011,1
1,10,Counter-Strike,2012,3
2,10,Counter-Strike,2013,7
3,10,Counter-Strike,2014,18
4,10,Counter-Strike,2015,15
...,...,...,...,...
3926,421630,A Study in Steampunk: Choice by Gaslight,2015,2
3927,421700,Sakura Santa,2015,1
3928,423120,Community College Hero: Trial by Fire,2015,2
3929,423880,Carpe Diem,2015,27


In [159]:
# Exportar dataframe a csv con el que voy a trabajar definitivamente la funcion 3

df_funcion3.to_csv('funcion3.csv',index=False)

In [162]:
def UsersRecommend( year : int ): # Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
    try:
        df_top3=df_funcion3[df_funcion3['year_review_posted']==year].nlargest(3,'recommend')
        return [{'Puesto 1': df_top3['app_name'].iloc[0]},{'Puesto 2:': df_top3['app_name'].iloc[1]},{'Puesto 3:': df_top3['app_name'].iloc[2]}]

    except Exception:
        return {'No existen datos para el valor ingresado'}
    
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

##### **Funcion 4** ______________________________________________________________________________________________________________________________________________________________________________

In [205]:
df_funcion4=pd.merge(df_user_reviews,df_user_items,on=['user_id','item_id'],how='left').merge(df_steam_games,on='item_id',how='left')
df_funcion4

,user_id,item_id,recommend,review_posted,year_review_posted,sentiment_analysis,items_count,steam_id,item_name,playtime_forever,playtime_2weeks,publisher,genres,app_name,release_date,price,developer,release_year
0,76561197970982479,1250,True,2011-11-05,2011,2,277.0,7.656120e+16,Killing Floor,10006.0,0.0,Tripwire Interactive,Action,Killing Floor,2009-05-14,19.99,Tripwire Interactive,2009.0
1,76561197970982479,22200,True,2011-07-15,2011,2,277.0,7.656120e+16,Zeno Clash,271.0,0.0,ACE Team,Action,Zeno Clash,2009-04-21,9.99,ACE Team,2009.0
2,76561197970982479,22200,True,2011-07-15,2011,2,277.0,7.656120e+16,Zeno Clash,271.0,0.0,ACE Team,Indie,Zeno Clash,2009-04-21,9.99,ACE Team,2009.0
3,76561197970982479,43110,True,2011-04-21,2011,2,277.0,7.656120e+16,Metro 2033,834.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,js41637,251610,True,2014-06-24,2014,2,888.0,7.656120e+16,Barbie™ Dreamhouse Party™,84.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108504,wayfeng,730,True,2015-10-14,2015,1,48.0,7.656120e+16,Counter-Strike: Global Offensive,42740.0,270.0,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,14.99,Valve,2012.0
108505,76561198251004808,253980,True,2015-10-10,2015,2,20.0,7.656120e+16,Enclave,1098.0,0.0,Topware Interactive,Rpg,Enclave,2003-03-10,4.99,Starbreeze,2003.0
108506,72947282842,730,True,2015-10-31,2015,0,2.0,7.656120e+16,Counter-Strike: Global Offensive,33.0,0.0,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,14.99,Valve,2012.0
108507,ApxLGhost,730,True,2015-12-14,2015,2,8.0,7.656120e+16,Counter-Strike: Global Offensive,10121.0,0.0,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,14.99,Valve,2012.0


In [206]:
# Seleccion de campos reelevantes para el desarrollo de la funcion 

df_funcion4=df_funcion4.loc[:,['item_id','recommend','year_review_posted','sentiment_analysis','item_name','app_name']]
df_funcion4


,item_id,recommend,year_review_posted,sentiment_analysis,item_name,app_name
0,1250,True,2011,2,Killing Floor,Killing Floor
1,22200,True,2011,2,Zeno Clash,Zeno Clash
2,22200,True,2011,2,Zeno Clash,Zeno Clash
3,43110,True,2011,2,Metro 2033,NaN
4,251610,True,2014,2,Barbie™ Dreamhouse Party™,NaN
...,...,...,...,...,...,...
108504,730,True,2015,1,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive
108505,253980,True,2015,2,Enclave,Enclave
108506,730,True,2015,0,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive
108507,730,True,2015,2,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive


In [207]:
# Armar data Frame filtrando los registros cuyos juegos fueron menos recomendados, para ello filtro el campo "recommend" por False y el campo "sentiment_analysis" para valores iguales a 0 (comentarios negativos)

df_funcion4=df_funcion4[(df_funcion4['recommend']==False) & (df_funcion4['sentiment_analysis']==0)]
df_funcion4=df_funcion4.reset_index(drop=True)
df_funcion4

,item_id,recommend,year_review_posted,sentiment_analysis,item_name,app_name
0,33440,False,2014,0,Driver San Francisco,NaN
1,359320,False,2015,0,Elite Dangerous,NaN
2,344760,False,2015,0,Reign Of Kings,Reign Of Kings
3,344760,False,2015,0,Reign Of Kings,Reign Of Kings
4,344760,False,2015,0,Reign Of Kings,Reign Of Kings
...,...,...,...,...,...,...
5796,311210,False,2015,0,Call of Duty: Black Ops III,Call of Duty®: Black Ops III
5797,311210,False,2015,0,Call of Duty: Black Ops III,Call of Duty®: Black Ops III
5798,305920,False,2015,0,Another Perspective,Another Perspective
5799,305920,False,2015,0,Another Perspective,Another Perspective


In [208]:
df_funcion4.isnull().sum()

item_id                  0
recommend                0
year_review_posted       0
sentiment_analysis       0
item_name             1501
app_name               618
dtype: int64

In [209]:
# unificar columna de nombre del juego 

df_funcion4['item_name'].fillna(df_funcion4['app_name'],inplace=True)
df_funcion4

,item_id,recommend,year_review_posted,sentiment_analysis,item_name,app_name
0,33440,False,2014,0,Driver San Francisco,NaN
1,359320,False,2015,0,Elite Dangerous,NaN
2,344760,False,2015,0,Reign Of Kings,Reign Of Kings
3,344760,False,2015,0,Reign Of Kings,Reign Of Kings
4,344760,False,2015,0,Reign Of Kings,Reign Of Kings
...,...,...,...,...,...,...
5796,311210,False,2015,0,Call of Duty: Black Ops III,Call of Duty®: Black Ops III
5797,311210,False,2015,0,Call of Duty: Black Ops III,Call of Duty®: Black Ops III
5798,305920,False,2015,0,Another Perspective,Another Perspective
5799,305920,False,2015,0,Another Perspective,Another Perspective


In [210]:
df_funcion4.isnull().sum()

item_id                 0
recommend               0
year_review_posted      0
sentiment_analysis      0
item_name             198
app_name              618
dtype: int64

In [211]:
# Eliminar registros nulos 

df_funcion4.dropna(subset=['item_name','app_name'],inplace=True)
df_funcion4.reset_index(drop=True)

,item_id,recommend,year_review_posted,sentiment_analysis,item_name,app_name
0,344760,False,2015,0,Reign Of Kings,Reign Of Kings
1,344760,False,2015,0,Reign Of Kings,Reign Of Kings
2,344760,False,2015,0,Reign Of Kings,Reign Of Kings
3,344760,False,2015,0,Reign Of Kings,Reign Of Kings
4,344760,False,2015,0,Reign Of Kings,Reign Of Kings
...,...,...,...,...,...,...
5178,311210,False,2015,0,Call of Duty: Black Ops III,Call of Duty®: Black Ops III
5179,311210,False,2015,0,Call of Duty: Black Ops III,Call of Duty®: Black Ops III
5180,305920,False,2015,0,Another Perspective,Another Perspective
5181,305920,False,2015,0,Another Perspective,Another Perspective


In [212]:
# Seleccionar columnas reelevantes para el desarrollo de la función

df_funcion4=df_funcion4.loc[:,['item_id','recommend','year_review_posted','item_name']]
df_funcion4

,item_id,recommend,year_review_posted,item_name
2,344760,False,2015,Reign Of Kings
3,344760,False,2015,Reign Of Kings
4,344760,False,2015,Reign Of Kings
5,344760,False,2015,Reign Of Kings
6,344760,False,2015,Reign Of Kings
...,...,...,...,...
5796,311210,False,2015,Call of Duty: Black Ops III
5797,311210,False,2015,Call of Duty: Black Ops III
5798,305920,False,2015,Another Perspective
5799,305920,False,2015,Another Perspective


In [213]:
df_funcion4=df_funcion4.groupby(by=['item_id','year_review_posted','item_name']).count()
df_funcion4=df_funcion4.reset_index()
df_funcion4



,item_id,year_review_posted,item_name,recommend
0,10,2015,Counter-Strike,1
1,20,2015,Team Fortress Classic,2
2,50,2014,Half-Life: Opposing Force,1
3,80,2014,Counter-Strike: Condition Zero,1
4,220,2014,Half-Life 2,4
...,...,...,...,...
876,410340,2015,Liftoff,5
877,412400,2015,GASP,6
878,417860,2015,Emily is Away,20
879,421770,2015,Pool Nation FX - Unlock Online,6


In [221]:
df_funcion4.to_csv('funcion4.csv',index=False) # pasar a csv 

In [222]:
def UsersNotRecommend( year : int ): # Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
    try:
        df_top3=df_funcion4[df_funcion4['year_review_posted']==year].nsmallest(3,'recommend')
        return [{'Puesto 1': df_top3['item_name'].iloc[0]},{'Puesto 2:': df_top3['item_name'].iloc[1]},{'Puesto 3:': df_top3['item_name'].iloc[2]}]

    except Exception:
        return {'No existen datos para el valor ingresado'}
 
# Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

##### **Funcion 5** ______________________________________________________________________________________________________________________________________________________________________________

In [272]:
# Unionr de tablas necesarias para el desarrollo de la funcion 
df_funcion5= pd.merge(df_user_reviews,df_steam_games,on='item_id',how='left')
df_funcion5

,user_id,item_id,recommend,review_posted,year_review_posted,sentiment_analysis,publisher,genres,app_name,release_date,price,developer,release_year
0,76561197970982479,1250,True,2011-11-05,2011,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,19.99,Tripwire Interactive,2009.0
1,76561197970982479,22200,True,2011-07-15,2011,2,ACE Team,Action,Zeno Clash,2009-04-21,9.99,ACE Team,2009.0
2,76561197970982479,22200,True,2011-07-15,2011,2,ACE Team,Indie,Zeno Clash,2009-04-21,9.99,ACE Team,2009.0
3,76561197970982479,43110,True,2011-04-21,2011,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,js41637,251610,True,2014-06-24,2014,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108503,wayfeng,730,True,2015-10-14,2015,1,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,14.99,Valve,2012.0
108504,76561198251004808,253980,True,2015-10-10,2015,2,Topware Interactive,Rpg,Enclave,2003-03-10,4.99,Starbreeze,2003.0
108505,72947282842,730,True,2015-10-31,2015,0,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,14.99,Valve,2012.0
108506,ApxLGhost,730,True,2015-12-14,2015,2,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,14.99,Valve,2012.0


In [273]:
df_funcion5=df_funcion5.loc[:,['sentiment_analysis','release_year']]
df_funcion5

,sentiment_analysis,release_year
0,2,2009.0
1,2,2009.0
2,2,2009.0
3,2,NaN
4,2,NaN
...,...,...
108503,1,2012.0
108504,2,2003.0
108505,0,2012.0
108506,2,2012.0


In [274]:
# Observación: Se observa que en el campo "release_year" existen valores nulos, esto se debe a que el dataframe "steam_games" no contiene de algunos juegos, esto se debe a que en el proceso de 
#limpieza y transformacion de datos del data set "steam_games" se perdieron algunos registros, por tanto , por cuestiones de practicidad se deciden eliminar aquellos registros con valores nulos 
df_funcion5.isnull().sum()

sentiment_analysis       0
release_year          8098
dtype: int64

In [275]:
# Eliminar registros qeu contengan valores ulos 

df_funcion5.dropna(subset='release_year',inplace=True)
df_funcion5

,sentiment_analysis,release_year
0,2,2009.0
1,2,2009.0
2,2,2009.0
5,0,2013.0
6,0,2013.0
...,...,...
108502,1,2012.0
108503,1,2012.0
108504,2,2003.0
108505,0,2012.0


In [276]:
# Exportar Dataset optimizado para la funcion 5
df_funcion5.to_csv('funcion5.csv',index=False)

In [279]:
def sentiment_analysis( year : int ): # Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
    df_año=df_funcion5[df_funcion5['release_year']==year]
    positivos=0
    negativos=0
    neutros=0
    for i in df_año['sentiment_analysis'].values:
        if i==0:
            negativos+=1
        elif i==1:
            neutros+=1
        elif i==2:
            positivos+=1

    return {'Negative': negativos, 'Neutral': neutros,'Positive':positivos}

# Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}